# Imports 

In [2]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt


# Processing

### Processing portfolio.csv

In [3]:
# creating dummies so that if I want, later I can change my prediction labels from offer_id to other attributes like reward, difficulty, or predict channels_
portfolio = pd.read_csv("portfolio.csv").drop("Unnamed: 0", axis=1)
# having universal names
portfolio.rename({"id": "offer_id"}, axis=1, inplace=True)
pd.get_dummies(portfolio)

,reward,difficulty,duration,"channels_['email', 'mobile', 'social']","channels_['web', 'email', 'mobile', 'social']","channels_['web', 'email', 'mobile']","channels_['web', 'email']",offer_type_bogo,offer_type_discount,offer_type_informational,offer_id_0b1e1539f2cc45b7b9fa7c272da2e1d7,offer_id_2298d6c36e964ae4a3e7e9706d1fb8c2,offer_id_2906b810c7d4411798c6938adc9daaa5,offer_id_3f207df678b143eea3cee63160fa8bed,offer_id_4d5c57ea9a6940dd891ad53e9dbe8da0,offer_id_5a8bc65990b245e5a138643cd4eb9837,offer_id_9b98b8c7a33c4b65b9aebfe6a799e6d9,offer_id_ae264e3637204a6fb9bb56bc8210ddfd,offer_id_f19421c1d4aa40978ebb69ca19b0e20d,offer_id_fafdcd668e3743c1bb461111dcafc2a4
0,10,10,7,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
1,10,10,5,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False
2,0,0,4,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False
3,5,5,7,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False
4,5,20,10,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False
5,3,7,7,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False
6,2,10,10,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
7,0,0,3,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False
8,5,5,5,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False
9,2,10,7,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False


### Processing profile.csv

In [4]:
profile = pd.read_csv("profile.csv").drop("Unnamed: 0", axis=1)

# having universal names
profile.rename({"id": "person"}, axis=1, inplace=True)

# removing age=118, considering it an outlier 
profile["age"] = profile["age"].replace(118, np.nan)

# Either all 3 attributed (income, gender, age) are present, or all 3 are absent together 
print("Times all 3 attributes are absent together:", ((profile["income"].isna() == profile["gender"].isna()) == (profile["age"].isna() == profile["gender"].isna())).mean())
print("Ratio of records where these 3 attributes are empty: ", profile["income"].isna().sum()/len(profile))

# Not including those users for now, adding an approach later to include them (not implemented)
profile = profile[profile["income"].notna()]


# I notice that different users have different date of joining and different incomes, and incomes usually increase every year 
# We Need to calculate how much all these users would be making today, or on the latest date in the dataset
# Also creating a new feature, days_diff, showing how long a user has been in the dataset
# If i wanted to do imputatiion by mean or median, it'd be more accurate now 
# assuming growth rate of 8% PA

annual_growth_rate = 0.08
daily_growth_rate = (1 + annual_growth_rate) ** (1/365) 

profile["became_member_on"] = pd.to_datetime(profile["became_member_on"], format="%Y%m%d")
profile['days_diff'] = (profile["became_member_on"].max() - profile['became_member_on']).dt.days
profile["max_date_income"] = profile["income"] * (daily_growth_rate) ** profile['days_diff']
profile.drop(["income"], axis=1, inplace=True)
profile["days_diff"] = profile["days_diff"] 


Times all 3 attributes are absent together: 1.0
Ratio of records where these 3 attributes are empty:  0.12794117647058822


### Processing transcript.csv

In [5]:
transcript = pd.read_csv("transcript.csv").drop("Unnamed: 0", axis=1)

# converting string to dict
transcript["value"] = transcript["value"].apply(lambda x: ast.literal_eval(x))

In [6]:
# Should've used df.apply with list flattening
# There are some entries where the dict in the value column has multiple values 
# Each key gets its own column for better clarity
# I also notice that one single user has multiple entries

new_transcript = []
for idx,i in transcript.iterrows():
    for key,value in i.value.items():
        new_i = i.copy()
        if key=="offer id":
            key="offer_id"
        
        new_i[key] = value
        new_i.value = key
        new_transcript += [new_i]
        
new_transcript = pd.DataFrame(new_transcript)
new_transcript.to_csv("new_transcript.csv", index=None)


In [7]:
new_transcript.head()

,person,event,value,time,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,offer_id,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,offer_id,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,offer_id,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,offer_id,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,offer_id,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


### Aggregating

In [8]:
# Mergeing new_transcript above with profile to have user
# I can also merge with portfolio var above, if my target label needs to change from offer_id to something else
# I'll only need focus on offer_id
# Shouldn't have created d_portfolio
mix = new_transcript.merge(profile, "inner", "person")

# I'll keep duplicates for now, can't be sure if these are added accidentally or are valid
# Can be an edge case where a user has been accidentally given the same offer/value twice, could do more analysis on this
# Do we want this to happen?
print("Ratio of duplicate entries", (1 - len(mix.drop_duplicates())/len(mix)))

# mix.drop_duplicates(inplace=True)

Ratio of duplicate entries 0.004115253304325606


In [18]:
mix.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [19]:
demo = mix.merge(portfolio, "left", "offer_id")
demo.drop("became_member_on",axis=1, inplace=True)

In [20]:
inputs = []
for key,value in demo.groupby("person").groups.items():
    demo2 = demo.iloc[value].copy()
    demo2 = demo2.sort_values("time").drop("person", axis=1)
    inputs.append(demo2)

In [21]:
analysis_dict = {}
for i in inputs:
    analysis = i["value"].to_list()
    for i in range(0, len(analysis)-1):
        if analysis[i] not in analysis_dict:
            analysis_dict[analysis[i]] = {}
        if analysis[i+1] not in analysis_dict[analysis[i]]:
            analysis_dict[analysis[i]][analysis[i+1]] = 0
        else:
            analysis_dict[analysis[i]][analysis[i+1]] += 1

In [22]:
# Trying to figure out what follows a particular event for a user
analysis_dict

{'offer_id': {'offer_id': 59797, 'amount': 55290, 'reward': 30622},
 'amount': {'offer_id': 62821, 'amount': 50159, 'reward': 1441},
 'reward': {'offer_id': 13659, 'amount': 16205, 'reward': 378}}

In [23]:
cols = []
for i,j in zip(demo.dtypes, demo.dtypes.index):
    if isinstance(i,np.dtypes.ObjectDType) and j!="person":
        cols+=[j]
cols

demo = pd.get_dummies(demo, columns=cols, dtype=float)
demo.columns

inputs = {}
for key,value in demo.groupby("person").groups.items():
    demo2 = demo.iloc[value].copy()
    demo2 = demo2.sort_values("time").drop("person", axis=1)
    inputs[key] = demo2

In [25]:
min([len(i) for i in list(inputs.values())]), max([len(i) for i in list(inputs.values())]) 

(2, 56)

In [26]:
import os
os.makedirs("transactions", exist_ok=True)
for person, df in inputs.items():
    df.to_csv(f"transactions/{person}.csv",  index=None)